In [1]:
import os
import pandas as pd

In [2]:
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [3]:
from loans.views import *

In [10]:
tenure = 12
principal = 52000
slug = 'A'
loan = Loan.objects.filter(tenure__gte=tenure,principal__gte=principal,slug=slug).order_by('tenure','principal').first()

In [15]:
list(LoanType.objects.get(loan_type='GPL').tenures.all())

[<Tenure: 3>,
 <Tenure: 12>,
 <Tenure: 6>,
 <Tenure: 0>,
 <Tenure: 60>,
 <Tenure: 24>]

In [7]:
loan = calculate_loan_details(Loan.objects.get(id='f5b4bd40-b865-11ea-be78-d0c63746edcc'), 30000, 9)

In [18]:
flt = Loan.objects.filter(tenure__gte=3,principal__lte=100000,slug='ing-ihtiyac-kredisi').order_by('tenure','principal')

In [9]:
tenure_ids = Loan.objects.filter(slug="is-bankasi-acil").values_list('tenure', flat=True)
tenures = Tenure.objects.filter(tenure__in=set(tenure_ids))

<QuerySet [<Tenure: 3>]>

In [32]:
from django.db.models import Min, Max

In [35]:
a = Loan.objects.filter(loan_type='GPL').aggregate(Min('principal'),Max('principal'))

In [39]:
import json

In [40]:
json.dumps(a)e

'{"principal__min": 0, "principal__max": 100000}'

In [20]:
pd.DataFrame(flt.values())

,active,bank_id,description,id,interest,loan_type_id,name,principal_id,slug,tenure_id,upd_program_id,upd_time,upd_user_id
0,True,ING Bank,ing ihtiyaç kredisi,14a0b47a-c111-11ea-b02d-d0c63746edcc,0.0096,GPL,İhtiyaç Kredisi,50000,ing-ihtiyac-kredisi,3,django_admin_panel,2020-07-08 11:49:29.316251+00:00,admin
1,True,ING Bank,ihtiyaç,a6fcaa6e-c5b7-11ea-a01b-d0c63746edcc,0.0054,GPL,ING İhtiyaç KREDİ,100000,ing-ihtiyac-kredisi,6,django_admin_panel,2020-07-14 09:51:55.971505+00:00,admin


In [15]:
flt.principal

<Principal: 100000>

In [9]:
loan.tenure.tenure

3

In [10]:
loan.principal.principal

30000

In [1]:
from scripts.py.payment_plan import payment_plan

In [17]:
loan.interest

'0.86%'

In [3]:
from babel.numbers import format_currency

In [11]:
format_currency(2434235, 'TRY', u'#,##0 ¤', locale='tr_TR').split(',')[0]

'2.434.235'

In [8]:
blogs = BlogPage.objects.all()

In [9]:
blogs_df = pd.DataFrame(list(BlogPage.objects.all().values()))

In [45]:
def payment_plan(amount, maturity, interest, bsmv, kkdf):
    total_interest = interest + interest * kkdf + interest * bsmv
    monthly_payment = (total_interest * amount) / (1 - (1 + total_interest) ** (-1 * maturity))
    payment_plan = pd.DataFrame(index=range(maturity), columns=['month','monthly_pay','principal_before','principal_paid','interest_paid','kkdf','bsmv','principal_left'])

    for i in range(maturity):
        if i == 0:
            principal_before = amount
        else:
            principal_before = payment_plan.loc[i-1, 'principal_left']

        interest_paid = principal_before * interest
        kkdf_paid = interest_paid * kkdf
        bsmv_paid = interest_paid * bsmv
        principal_paid = monthly_payment - interest_paid - kkdf_paid - bsmv_paid
        principal_left = principal_before - principal_paid

        payment_plan.loc[i] = [i+1,monthly_payment, principal_before, principal_paid, interest_paid, kkdf_paid, bsmv_paid, principal_left]
        payment_plan = payment_plan.astype('float')

    payment_plan.drop('principal_before', axis=1, inplace=True)
    payment_plan = payment_plan.round(0)
    payment_plan.iloc[-1,-1]=0
    payment_plan.columns=['Ay','Taksit','Anapara','Faiz','KKDF','BSMV','Kalan Ödeme']
    return payment_plan


In [46]:
df = payment_plan(30000,30,0.003,0.005,0.005)

In [52]:
df.shape[0]

30

In [48]:
for i, j in df.iterrows():
    print("i: "+str(i))
    print("j: "+str(j))
    print("----")

i: 0
j: Ay                 1.0
Taksit          1048.0
Anapara          957.0
Faiz              90.0
KKDF               0.0
BSMV               0.0
Kalan Ödeme    29043.0
Name: 0, dtype: float64
----
i: 1
j: Ay                 2.0
Taksit          1048.0
Anapara          960.0
Faiz              87.0
KKDF               0.0
BSMV               0.0
Kalan Ödeme    28084.0
Name: 1, dtype: float64
----
i: 2
j: Ay                 3.0
Taksit          1048.0
Anapara          963.0
Faiz              84.0
KKDF               0.0
BSMV               0.0
Kalan Ödeme    27121.0
Name: 2, dtype: float64
----
i: 3
j: Ay                 4.0
Taksit          1048.0
Anapara          965.0
Faiz              81.0
KKDF               0.0
BSMV               0.0
Kalan Ödeme    26156.0
Name: 3, dtype: float64
----
i: 4
j: Ay                 5.0
Taksit          1048.0
Anapara          968.0
Faiz              78.0
KKDF               0.0
BSMV               0.0
Kalan Ödeme    25187.0
Name: 4, dtype: float64
----
i: 5
j: Ay

In [50]:
len(df.index)

30

In [8]:
from datetime import datetime as dtm
from datetime import date as dt
from datetime import timedelta

In [11]:
dt.today()+timedelta(days=-1)

datetime.date(2020, 8, 10)

In [5]:
dt.today() if dtm.now().hour>5 else dt.today()+timedelta(days=-1)

0